#  네이버 뉴스 가져오기

2020년1월1일 ~ 2020년 4월 14일 데이터 가져오기

'기사 더보기' 누르기 전 -> 기사가 몇개 없음

=> 동적으로 변경

In [1]:
import urllib.request 
from bs4 import BeautifulSoup # 특정 웹 페이지의 HTML을 스크랩하여 데이터를 수집할 때 사용
import pandas as pd
import numpy as np
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# 날짜 가져오기
# 시작 날짜와 끝 날짜 설정

start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 4, 14)

In [ ]:
# 웹 드라이버 시작
driver = webdriver.Chrome()
df_2020 = pd.DataFrame()

# 네이버 뉴스 페이지로 이동
current_date = start_date
while current_date <= end_date:
    formatted_date = current_date.strftime("%Y%m%d")

    driver.get('https://news.naver.com/breakingnews/section/100/265?date={}'.format(formatted_date))

    # '기사 더보기'없을 때까지
    try:
        while True:
            elem1 = driver.find_element(By.CLASS_NAME, "section_more_inner._CONTENT_LIST_LOAD_MORE_BUTTON")
            elem1.click()
            time.sleep(1)  # 대기
    except Exception as e:
        print(e)


    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup을 사용하여 페이지 소스를 파싱
    soup = BeautifulSoup(page_source, 'html.parser')

    # 제목 가져오기
    titles = soup.select('div.section_latest > div > div.section_latest_article._CONTENT_LIST._PERSIST_META > \
                              div > ul > li > div > div > div.sa_text > a > strong.sa_text_strong')

    # 링크 가져오기
    links = soup.select('div.section_latest > div > div.section_latest_article._CONTENT_LIST._PERSIST_META > \
                              div > ul > li > div > div > div.sa_text > a')

    title_lst = []
    content_lst = []

    # 링크로 이동해서 내용 가져오기
    for title, link in tqdm(zip(titles, links)):
        article_link = link['href'] # 링크 가져오기
        driver.get(article_link) # 링크로 이동

        # 페이지 소스 가져오기
        article_page_source = driver.page_source
        article_soup = BeautifulSoup(article_page_source, 'lxml')

        # 내용 가져오기
        content = article_soup.select_one('div.ct_wrap > div.ct_scroll_wrapper > div#newsct > div#ct > div#contents > \
                            div#newsct_article > article#dic_area')

        if content:
            content_lst.append(content.text.strip())
        else:
            content_lst.append('No content available') # 내용이 없는 경우 예외 처리

        title_lst.append(title.string)


    # 데이터 저장
    df = pd.DataFrame({'title' : title_lst, 'content' : content_lst})
    df_2020 = pd.concat([df_2020,df], ignore_index=True)
    
    current_date += datetime.timedelta(days=1)

# 웹 드라이버 종료
driver.quit()

df_2020.to_csv('news_2020.csv', index = False)

In [ ]:
df_news = pd.read_csv('./data/news_2020.csv')

df_news